In [8]:

# Config:
# Without train_test_split()
# Feature - real, imaginary, magnitude
# Sample - 100 CIRs
# Non-zero coefficients - 10
# -------------------------------------------------------------------------------------------------------
import numpy as np  # This helps us work with arrays (lists of numbers).
from sklearn.linear_model import OrthogonalMatchingPursuit  # This helps us find sparse representations.

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)


# Imagine we have some data from two people: Alice (the good person) and Eve (the bad person).
# Each has some channel data that we can use to tell them apart.

# Let's pretend we have 5 samples (Channel Impulse Responses or CIRs) from Alice.
# Each sample has 251 points, and each point has a real and an imaginary part.

# Create random data for Alice.
alice_CIRs = np.random.rand(5, 251, 2)  # This makes random numbers between 0 and 1.

# Create random data for Eve.
eve_CIRs = np.random.rand(5, 251, 2)  # Same as above, but for Eve.

alice_real = alice_CIRs[:, :, 0]
alice_imag = alice_CIRs[:, :, 1]
alice_magnitude = np.abs(alice_real + 1j * alice_imag)
alice_features = np.hstack((alice_real, alice_imag, alice_magnitude))

# EVE features - real, imaginary, magnitude
eve_real = eve_CIRs[:, :, 0]
eve_imag = eve_CIRs[:, :, 1]
eve_magnitude = np.abs(eve_real + 1j * eve_imag)
eve_features = np.hstack((eve_real, eve_imag, eve_magnitude))

In [9]:
# Now, we want to make a dictionary with examples from both Alice and Eve.
# This dictionary will help us classify new signals.

# First, we need to turn each CIR (which is 2D) into a 1D array (a long list of numbers).
# We do this by reshaping.

# Reshape Alice's data.
alice_atoms = alice_features.reshape(5, -1)  # '-1' means we flatten the last two dimensions.

# Reshape Eve's data.
eve_atoms = eve_features.reshape(5, -1)  # Do the same for Eve.

print(alice_atoms.shape)  # This should be (5, 502).
print(eve_atoms.shape)  # This should be (5, 502).

(5, 753)
(5, 753)


In [10]:
# Now, we combine Alice's and Eve's data into one big dictionary.
dictionary = np.vstack((alice_atoms, eve_atoms))  # 'vstack' stacks them vertically.
print(dictionary.shape)  # This should be (10, 502).

(10, 753)


In [11]:
# But we need the dictionary to have atoms (examples) as columns, so we transpose it.
dictionary = dictionary.T  # This flips the array so rows become columns.

print(dictionary.shape)  # This should be (502, 10).

(753, 10)


In [12]:
# Let's say we have a new signal (test_signal) that we want to classify.
# We'll use one of Alice's samples for this example.
test_signal = alice_CIRs[0].reshape(-1)  # Flatten the first sample from Alice.
test_real = alice_CIRs[0, :, 0]
test_imag = alice_CIRs[0, :, 1]
test_magnitude = np.abs(test_real + 1j * test_imag)
test_features = np.hstack((test_real, test_imag, test_magnitude))
print(test_features.shape)  # This should be (753,).

(753,)


In [13]:
# Now, we want to represent this test signal using the dictionary.
# We use Orthogonal Matching Pursuit (OMP) to find the sparse coefficients.
# Sparse means that most coefficients will be zero; only a few will be non-zero.

# Create an OMP model that allows up to 2 non-zero coefficients.
omp = OrthogonalMatchingPursuit(n_nonzero_coefs=2)

# Fit the model to find the coefficients.
omp.fit(dictionary, test_features)

# Get the coefficients from the model.
coefficients = omp.coef_
print(coefficients)  # This should be a list of 10 numbers, mostly zeros.

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/utils/_param_validation.py:186: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  return func(*args, **kwargs)


In [14]:
# Start with an empty list to store residuals.
residuals = []
coef_class = np.zeros_like(coefficients)
print(coefficients)
# We have two classes: Alice (0) and Eve (1).
for i in range(2):
    # Create an array of zeros like the coefficients.
    coef_class = np.zeros_like(coefficients)
    if i == 0:
        # For Alice, keep her coefficients (first 5), set Eve's to zero.
        coef_class[:5] = coefficients[:5]
        print(coef_class)
    else:
        # For Eve, keep her coefficients (last 5), set Alice's to zero.
        coef_class[5:] = coefficients[5:]
        print(coef_class)
    # Reconstruct the signal using only the coefficients from one class.
    reconstructed_signal = dictionary @ coef_class
    # Calculate the residual (difference between test signal and reconstructed signal).
    residual = np.linalg.norm(test_features - reconstructed_signal)
    # Add the residual to the list.
    residuals.append(residual)

# Find which class has the smallest residual.
predicted_class = np.argmin(residuals)

# Now, we print out which class the test signal belongs to.
if predicted_class == 0:
    print("The test signal is classified as Alice.")
else:
    print("The test signal is classified as Eve.")

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
The test signal is classified as Alice.
